<a href="https://colab.research.google.com/github/KairaNithin/XML-Experiments/blob/main/code_mixed_Data_Creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import csv
import re

In [2]:
!pip install ekphrasis
from ekphrasis.classes.segmenter import Segmenter
# to leverage word statistics from Twitter
seg_tw = Segmenter(corpus = "twitter")

Reading twitter - 1grams ...
Reading twitter - 2grams ...


/usr/local/lib/python3.7/dist-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


In [3]:
!pip install tweet-preprocessor
import preprocessor as tweet_proc

In [4]:
!pip install emot
from emot.emo_unicode import UNICODE_EMO, EMOTICONS

In [5]:
def make_list(proc_obj):
  if proc_obj == None:
    return []
  
  store = []
  for unit in proc_obj:
    store.append(unit.match)
  
  return store

In [6]:
def emotext(text):
    for emot in UNICODE_EMO:
        text = text.replace(emot, "_".join(UNICODE_EMO[emot].replace(",", "").replace(":", "").split()))
    return text

In [7]:
tweets = []
raw_tweet_texts = []

tweet_ids = []

hashtags = []
smileys = []
emojis = []
urls = []
mentions = []
numbers = []
reserveds = []

task_1_labels = []

In [8]:
import pandas as pd
hi_en_df = pd.read_csv("hindi_english.csv")

In [9]:
for i in range(len(hi_en_df)):

    if hi_en_df['task1'][i] == "no":
        task_1_labels.append("NOT")
        tweets.append(hi_en_df['text'][i])
        raw_tweet_texts.append(tweet_proc.clean(hi_en_df['text'][i]))

        parse_obj = tweet_proc.parse(hi_en_df['text'][i])

        hashtags.append(make_list(parse_obj.hashtags))
        smileys.append(make_list(parse_obj.smileys))
        emojis.append(make_list(parse_obj.emojis))
        urls.append(make_list(parse_obj.urls))
        mentions.append(make_list(parse_obj.mentions))
        numbers.append(make_list(parse_obj.numbers))
        reserveds.append(make_list(parse_obj.reserved))
        tweet_ids.append(i)

    if hi_en_df['task1'][i] == "yes":
      task_1_labels.append("HOF")
      tweets.append(hi_en_df['text'][i])
      raw_tweet_texts.append(tweet_proc.clean(hi_en_df['text'][i]))

      parse_obj = tweet_proc.parse(hi_en_df['text'][i])

      hashtags.append(make_list(parse_obj.hashtags))
      smileys.append(make_list(parse_obj.smileys))
      emojis.append(make_list(parse_obj.emojis))
      urls.append(make_list(parse_obj.urls))
      mentions.append(make_list(parse_obj.mentions))
      numbers.append(make_list(parse_obj.numbers))
      reserveds.append(make_list(parse_obj.reserved))
      tweet_ids.append(i)

In [10]:
emoji_texts = []

for emo_list in emojis:
  texts = []
  for emoji in emo_list:
    text = emotext(emoji)
    texts.append(text)
  emoji_texts.append(texts)

In [11]:
segmented_hashtags = []

for hashset in hashtags:
  segmented_set = []
  for tag in hashset:
    word = tag[1: ]
    # removing the hash symbol
    segmented_set.append(seg_tw.segment(word))
  segmented_hashtags.append(segmented_set)

In [12]:
import pickle
name = 'hi_en_train.pickle'
dickie = {}
dickie['tweet_id'] = tweet_ids
dickie['task_1'] = task_1_labels
dickie['full_tweet'] = tweets
dickie['tweet_raw_text'] = raw_tweet_texts
dickie['hashtags'] = hashtags
dickie['smiley'] = smileys
dickie['emoji'] = emojis
dickie['url'] = urls
dickie['mentions'] = mentions
dickie['numerals'] = numbers
dickie['reserved_word'] = reserveds
dickie['emotext'] = emoji_texts
dickie['segmented_hash'] = segmented_hashtags



with open(name, 'wb') as f:
  pickle.dump(dickie, f)

In [14]:
with open("hi_en_train.pickle", 'rb') as f:
    hi_en = pickle.load(f)
hi_en = pd.DataFrame.from_dict(hi_en)

In [15]:
hi_en

,tweet_id,task_1,full_tweet,tweet_raw_text,hashtags,smiley,emoji,url,mentions,numerals,reserved_word,emotext,segmented_hash
0,0,NOT,Knowing ki Vikas kitna samjhata hai Priyanka a...,Knowing ki Vikas kitna samjhata hai Priyanka a...,[],[],[],[],[],[],[],[],[]
1,1,NOT,I am Muhajir .. Aur mere lye sab se Pehly Paki...,I am Muhajir .. Aur mere lye sab se Pehly Paki...,"[#Muhajir, #Pakistani]",[],[],[],[],[ 10],[],[],"[muhajir, pakistani]"
2,2,NOT,Doctor sab sahi me ke PhD (in hate politics) ...,Doctor sab sahi me ke PhD (in hate politics) w...,[],[],[],[pic.twitter.com/fk1qUbQstw],[],[],[],[],[]
3,3,NOT,Poore Desh me Patel OBC me aate Hain sirf gujr...,Poore Desh me Patel OBC me aate Hain sirf gujr...,[],[],[],[],[],[],[],[],[]
4,4,HOF,Sarkar banne ke bad Hindu hit me ek bhi faisla...,Sarkar banne ke bad Hindu hit me ek bhi faisla...,[],[],[],[],[],[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4569,4573,NOT,ye attankwadi Indian agent hai jo terrorism ph...,ye attankwadi Indian agent hai jo terrorism ph...,[],[],[],[],[],[],[],[],[]
4570,4574,NOT,bola na terrorism ko support karna band karoge...,bola na terrorism ko support karna band karoge...,[],[],[],[],[],[],[],[],[]
4571,4575,NOT,lagta hai aap ne movie dekhi hai which is writ...,lagta hai aap ne movie dekhi hai which is writ...,[],[],[],[],[],[],[],[],[]
4572,4576,NOT,tum log terrorism ko support karna band kardo ...,tum log terrorism ko support karna band kardo ...,[],[],[],[],[],[],[],[],[]
